In [1]:
import numpy as np
import pandas as pd
import time
from scipy.integrate import solve_ivp

### Length estimation (mm)

In [2]:
# Betini et al. (2019)
def Size_estimation(age, temperature=22, food='high'):
    # T = 15 o C
    a_low_15 = 0.354
    b_low_15 = 0.527
    a_high_15 = 0.105
    b_high_15 = 0.953
  
    # T = 25 o C
    a_low_25 = 0.811
    b_low_25 = 0.355
    a_high_25 = 0.698
    b_high_25 = 0.83
    
    if food == 'low':
        if temperature <= 15:
            a = a_low_15
            b = b_low_15
        elif temperature >= 25:
            a = a_low_25
            b = b_low_25
        else:
            a = np.interp(temperature, np.array([15,25]), np.array([a_low_15, a_low_25]))
            b = np.interp(temperature, np.array([15,25]), np.array([b_low_15, b_low_25]))
    elif food == 'high':
        if temperature <= 15:
            a = a_high_15
            b = b_high_15
        elif temperature >= 25:
            a = a_high_25
            b = b_high_25
        else:
            a = np.interp(temperature, np.array([15,25]), np.array([a_high_15, a_high_25]))
            b = np.interp(temperature, np.array([15,25]), np.array([b_high_15, b_high_25]))
    return a + b * np.log(age)

### Filtration rate estimation (ml/h)

In [3]:
def Filtration_rate_estimation(Length, Temperature=22, method="Preuss"):
  
  if method == "Burns":
    # Filtering rate of Daphnia magna is calculated based on Burns et al. 1969.
    # Input: Length [mm], Temperature [oC]/ Output: filtration rate[mL/h]
    F_rate_15 = 0.153 * Length**2.16
    F_rate_20 = 0.208 * Length**2.80
    F_rate_25 = 0.202 * Length**2.38
    
    if Temperature <= 15:
      F_rate = F_rate_15
    elif Temperature >= 25:  
      F_rate = F_rate_25
    else: 
      F_rate = np.interp(Temperature, [15,20,25], [F_rate_15, F_rate_20, F_rate_25])
  elif method == "Preuss":
    F_rate = 0.5 * Length**2.45
  else:
    raise ValueError("Please select a valid estimation method; either 'Burns' or 'Preuss'")
  return F_rate

### Dry weight estimation (mg)

In [4]:
# Dumont et al. (1975)
# Input: length [mm]/ Output: dry weight[mg]
def dry_weight_estimation(L):
    w1 = (1.89e-06*(L*1000)**2.25)/1000 #Donka Lake
    w2 = (4.88e-05*(L*1000)**1.80)/1000 #River Sambre
    # Selected w1 after validation with Martin-Creuzburg et al. (2018
    return(w1)

### V_water and sedimentation rates

In [5]:
# V_water is the volume of water (in L) in the corresponding experiment.
# The volume remains constant during the experiment and equal to 100 ml
V_water = 0.1 # L

# Load the predicted ksed values
ksed_predicted = pd.read_csv("C:/Users/vassi/Documents/GitHub/TiO2_aggregation_and_uptake/Exposure/Fan_2016/data/Fan_2016_ksed_predictions.csv")
ksed_predicted = ksed_predicted.iloc[:, [0, 3, 5]]
ksed_predicted.columns = ["Name", "Concentration_mg/L", "k_sed"]
# We will keep only the three ksed values for T1 TiO2 for the three different concentrations
ksed_predicted = ksed_predicted.iloc[:3, :]

### ODE system

In [6]:
def ode_func(t, inits, params):
    
    # Units explanation:
    # C_water: mg TiO2 / L water (same as data)
    # C_daphnia: mg TiO2 / mg daphnia 
    # k_sed: 1/h
    # F_rate: L water/h/individual daphnia 
    # ke_2: 1/h
    
    # set the initial conditions
    C_water, C_daphnia, M_Daphnia_excreted, M_sed = inits
    # Assign values to the parameters
    age, L, dry_weight, F_rate, a, ke_2, k_sed, C_sat, n, V_water = params
    
    # Number of Daphnids per beaker
    N_current = 10
    # Fan et al. (2016) let the daphnids in SM7 medium for 3 hours
    # prior to the experiment to empty their guts
    time_d = t/24 #time in days
    # Weight loss under starvation through exponential decay (Elen et al., 1989)
    reduction = np.where(time_d<2, (1- 0.32*(1-np.exp(-36.6*time_d))),(0.682 - 0.435 * (1- 16.79*np.exp(-1.41*time_d))))

    dry_weight_current = reduction * dry_weight
    # C_water: TiO2 concentration in water

    if t == 10: 
        dC_water = 10 -(N_current*a*(F_rate/1000)*((1-C_daphnia/C_sat)**n)*C_water)/V_water - k_sed*C_water
    else:
        dC_water = -(N_current*a*(F_rate/1000)*((1-C_daphnia/C_sat)**n)*C_water)/V_water - k_sed*C_water
    
    # Daphnia magna
    dC_daphnia = a*(F_rate/1000)*((1-C_daphnia/C_sat)**n)*C_water/dry_weight - ke_2*C_daphnia 
    
    # Excreted from each D.magna
    dM_Daphnia_excreted =  N_current*ke_2*C_daphnia*dry_weight  
    
    # Mass in  Sediment
    dM_sed = k_sed*C_water*V_water
    
    # TiO2 mass in all D.magna
    M_daphnia_tot = C_daphnia*dry_weight*N_current
    
    # TiO2 mass in water
    M_water = C_water*V_water
    
    # Mass balance of TiO2 (should always be the total mass of the system)
    Mass_balance = M_daphnia_tot + M_water + M_sed + M_Daphnia_excreted
    
    return[dC_water, dC_daphnia, dM_Daphnia_excreted, dM_sed]

### Input 

In [7]:
inits = [10,0,0,0]

# Calculate the physiological parameters
age = 14
L = Size_estimation(age) #mm
dry_weight =  dry_weight_estimation(L) #mg
F_rate = Filtration_rate_estimation(L, method = 'Preuss')#mL/h
#print([age, L, dry_weight, F_rate])

params = {'age':age,
          'L':L,
          'dry_weight':dry_weight,
          'F_rate':F_rate,
          'a':0.331,
          'ke_2':0.021,
          'k_sed':ksed_predicted.iloc[2][2],
          'C_sat':0.136, 
          'n':2,
          'V_water': V_water}

print(params)
t_span = (0, 48)
t_eval = np.arange(0, 49, 1)


{'age': 14, 'L': 2.8078987990434676, 'dry_weight': 0.10847260479315352, 'F_rate': 6.2734228224762525, 'a': 0.331, 'ke_2': 0.021, 'k_sed': 0.18366, 'C_sat': 0.136, 'n': 2, 'V_water': 0.1}


In [8]:
start_time = time.time()
solution = solve_ivp(fun=ode_func, t_span=t_span, method='LSODA', t_eval=t_eval, 
                     atol = 1e-05, rtol = 1e-05,
                     y0=inits, args=[list(params.values())])
finish_time = time.time()
solution_time = (finish_time - start_time)


In [9]:
colnames = ['Time', 'C_water','C_daphnia','M_daphnia_excreted','M_sed']

solution_df = pd.DataFrame(data=np.column_stack((solution.t, solution.y.T)), columns = colnames )
solution_df

,Time,C_water,C_daphnia,M_daphnia_excreted,M_sed
0,0.0,10.000000,0.000000,0.000000,0.000000
1,1.0,7.599338,0.073971,0.001101,0.158727
2,2.0,6.134976,0.091568,0.003019,0.284158
3,3.0,5.013220,0.098965,0.005199,0.386129
4,4.0,4.115051,0.102636,0.007500,0.469662
5,5.0,3.384681,0.104509,0.009862,0.538307
6,6.0,2.786561,0.105366,0.012254,0.594797
7,7.0,2.294959,0.105587,0.014657,0.641314
8,8.0,1.890078,0.105378,0.017061,0.679626
9,9.0,1.556223,0.104862,0.019456,0.711175


In [10]:
inits = [10,0,0,0]
t_span = (0, 10)
t_eval = np.arange(0, 11, 1)
colnames = ['Time', 'C_water','C_daphnia','M_daphnia_excreted','M_sed']
        
for i in range(3):
    solution = solve_ivp(fun=ode_func, t_span=t_span, method='BDF', t_eval=t_eval,
                             atol = 1e-05, rtol = 1e-05,
                             y0=inits, args=[list(params.values())])
        
    if i == 0:
        total_df = pd.DataFrame(data=np.column_stack((solution.t, solution.y.T)), columns = colnames )
    else:
        new_df = pd.DataFrame(data=np.column_stack((solution.t, solution.y.T)), columns = colnames )
        total_df = pd.concat([total_df, new_df.iloc[1:]], axis=0)
        
    t_span = (t_span[0]+10, t_span[1]+10)
    t_eval = np.arange(t_span[0], t_span[1]+1, 1)
    inits = solution.y.T[-1]
    inits[0] = 10

total_df

,Time,C_water,C_daphnia,M_daphnia_excreted,M_sed
0,0.0,10.000000,0.000000,0.000000,0.000000
1,1.0,7.599312,0.073973,0.001100,0.158727
2,2.0,6.134951,0.091570,0.003019,0.284158
3,3.0,5.013196,0.098967,0.005199,0.386129
4,4.0,4.115025,0.102639,0.007500,0.469662
5,5.0,3.384649,0.104512,0.009862,0.538307
6,6.0,2.786526,0.105369,0.012254,0.594797
7,7.0,2.294930,0.105589,0.014657,0.641314
8,8.0,1.890039,0.105381,0.017061,0.679626
9,9.0,1.556193,0.104865,0.019456,0.711175


### A custom function to solve the ODEs using events

In [11]:
def ode_events(model, inits, t_span, time_step, t_events, y_events):
    # column names for the final dataframe with the solutions
    colnames = ['Time', 'C_water','C_daphnia','M_daphnia_excreted','M_sed']
    # assign the initial and the final time point of the integration
    t_i, t_f = t_span[0], t_span[1]
    
    for i in range(len(t_events)+1):
        
        if i == 0: # for 0 < t < t_events[0]
            t_span = (t_i, t_events[i])
        elif i == len(t_events):
            t_span = (t_events[i-1], t_f) # for t_events[final] < t < t_f  (till the end of experiment)  
        else:
            t_span = (t_events[i-1], t_events[i]) # time intervals between the events
            
        t_eval = np.arange(t_span[0], t_span[1]+time_step/2, time_step) # set the integration time points based on the current t_span
        t_eval[-1] = t_span[1] # correct the final point due to floating errors
        
        solution = solve_ivp(fun=model, t_span=t_span, method='BDF', t_eval=t_eval,
                                 atol = 1e-05, rtol = 1e-05,
                                 y0=inits, args=[list(params.values())])

        if i == 0:
            total_df = pd.DataFrame(data=np.column_stack((solution.t, solution.y.T)), columns = colnames ) # keep the solution in df 
        else:
            new_df = pd.DataFrame(data=np.column_stack((solution.t, solution.y.T)), columns = colnames )
            total_df = pd.concat([total_df, new_df.iloc[1:]], axis=0, ignore_index=True) # concatenate the solutions from the previous time spans with the new solution
        
        inits = solution.y.T[-1] # define the initial values for the next time span
        if i < len(t_events):
            inits[0] = y_events[i] # Adjust the C_water value 
    return total_df    

In [13]:
inits = [10,0,0,0] # initial values of state variables
t_span = (0, 50) # time span of integration
t_events = [10, 20, 30, 35] # time points of C_water refreshment 
y_events = [10, 20, 10, 5] # C_water that was achieved at t_events
time_step = 1

solution = ode_events(ode_func, inits, t_span, time_step, t_events, y_events)
solution

,Time,C_water,C_daphnia,M_daphnia_excreted,M_sed
0,0.0,10.000000,0.000000,0.000000,0.000000
1,1.0,7.599312,0.073973,0.001100,0.158727
2,2.0,6.134951,0.091570,0.003019,0.284158
3,3.0,5.013196,0.098967,0.005199,0.386129
4,4.0,4.115025,0.102639,0.007500,0.469662
5,5.0,3.384649,0.104512,0.009862,0.538307
6,6.0,2.786526,0.105369,0.012254,0.594797
7,7.0,2.294930,0.105589,0.014657,0.641314
8,8.0,1.890039,0.105381,0.017061,0.679626
9,9.0,1.556193,0.104865,0.019456,0.711175
